In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from PIL import Image
import os
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from tqdm import tqdm

class AnemiaDataset(Dataset):
    def __init__(self, data_dir=r"anemia_project\anemia_detection" , transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []
        
        # Load all image paths and labels from anemia_detection folder
        for filename in os.listdir(data_dir):
            if filename.lower().startswith('anemic'):
                self.images.append(os.path.join(data_dir, filename))
                self.labels.append(1)  # Anemic class
            elif filename.lower().startswith('non-anemic'):
                self.images.append(os.path.join(data_dir, filename))
                self.labels.append(0)  # Non-Anemic class
        
        print(f"Found {len(self.images)} images in total")
        print(f"Anemic samples: {sum(self.labels)}")
        print(f"Non-Anemic samples: {len(self.labels) - sum(self.labels)}")
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

class AnemiaNet(nn.Module):
    def __init__(self, num_classes=2):
        super(AnemiaNet, self).__init__()
        # Use ResNet18 as backbone
        self.model = models.resnet18()
        
        # Freeze early layers
        for param in list(self.model.parameters())[:-4]:
            param.requires_grad = False
            
        # Modify the final layer for binary classification
        num_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, num_classes)
        )
        
    
    def forward(self, x):
        return self.model(x)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device='cuda'):
    best_val_acc = 0.0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Calculate total batches for progress bar
        total_batches = len(train_loader)
        
        # Use tqdm with total number of batches
        pbar = tqdm(enumerate(train_loader), total=total_batches, 
                   desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for batch_idx, (images, labels) in pbar:
            try:
                # Move data to device
                images = images.to(device)
                labels = labels.to(device)
                
                # Forward pass
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                # Backward pass
                loss.backward()
                optimizer.step()
                
                # Calculate accuracy
                running_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                # Update progress bar with current loss and accuracy
                avg_loss = running_loss / (batch_idx + 1)
                acc = 100. * correct / total
                pbar.set_postfix({
                    'loss': f'{avg_loss:.3f}',
                    'acc': f'{acc:.2f}%'
                })
                
            except RuntimeError as e:
                if "out of memory" in str(e):
                    if hasattr(torch.cuda, 'empty_cache'):
                        torch.cuda.empty_cache()
                    print(f"\nWARNING: GPU out of memory in batch {batch_idx}. Skipping batch...")
                    continue
                else:
                    raise e
            except Exception as e:
                print(f"\nError in batch {batch_idx}: {str(e)}")
                continue
        
        # Calculate training metrics
        train_acc = 100. * correct / total
        train_loss = running_loss / len(train_loader)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        print("\nRunning validation...")
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        val_acc = 100. * correct / total
        val_loss = val_loss / len(val_loader)
        
        print(f'\nEpoch {epoch+1}/{num_epochs} Results:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_anemia_model.pth')
            print(f'New best model saved! Validation Accuracy: {val_acc:.2f}%')

    print("\nTraining completed!")
    print(f"Best validation accuracy: {best_val_acc:.2f}%")
    return model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Data augmentation and preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create dataset
full_dataset = AnemiaDataset(transform=transform)

# Split dataset into train and validation
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

print(f"\nSplit sizes:")
print(f"Training set: {len(train_dataset)} images")
print(f"Validation set: {len(val_dataset)} images")

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Initialize model
model = AnemiaNet().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device=device)



In [4]:
import os
import torch
import torchvision
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Define custom dataset class
class AnemiaDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Load image filenames and their corresponding labels
        for filename in os.listdir(data_dir):
            if filename.endswith(('.jpg', '.jpeg', '.png')):  # Check for image files
                # Check if the image name starts with "Anemic" or "Non-Anemic"
                if filename.startswith("Anemic"):
                    label = 1  # Anemic
                elif filename.startswith("Non-Anemic"):
                    label = 0  # Non-Anemic
                else:
                    continue  # Skip images with unexpected filenames
                
                # Append the full path and label
                self.image_paths.append(os.path.join(data_dir, filename))
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        # Load the image
        image = Image.open(img_path).convert('RGB')  # Ensure 3 channels (RGB)

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        return image, label

# Define the transform to resize and normalize images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ImageNet normalization
])

# Define the path to the data folder
dataset_dir = r"anemia_project\anemia_detection\Palm"

# Create custom dataset
dataset = AnemiaDataset(data_dir=dataset_dir, transform=transform)

# Split dataset into training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create DataLoader for training and validation
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
from torchvision import models

# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer for binary classification (2 classes)
model.fc = nn.Linear(model.fc.in_features, 1)  # Output size = 1 for binary classification
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available


In [6]:
# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # For binary classification
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
# Function to train the model
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            outputs = outputs.squeeze()  # Remove extra dimensions for binary classification
            loss = criterion(outputs, labels.float())  # Compute loss
            loss.backward()  # Backpropagate

            # Update weights
            optimizer.step()

            # Compute accuracy
            predicted = torch.round(torch.sigmoid(outputs))  # Get predictions
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            running_loss += loss.item()

        # Calculate training accuracy and loss
        train_accuracy = correct / total
        train_loss = running_loss / len(train_loader)

        # Evaluate on validation data
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')
                outputs = model(inputs)
                outputs = outputs.squeeze()

                # Compute loss and accuracy
                loss = criterion(outputs, labels.float())
                val_loss += loss.item()

                predicted = torch.round(torch.sigmoid(outputs))
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)

        val_accuracy = val_correct / val_total
        val_loss = val_loss / len(val_loader)

        # Print results for this epoch
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

        # Save the best model based on validation accuracy
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pth')

    print("Training complete.")
    print(f"Best Validation Accuracy: {best_accuracy:.4f}")

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)


In [ ]:
# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Evaluate the model on the validation set
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')
        outputs = model(inputs)
        outputs = outputs.squeeze()

        predicted = torch.round(torch.sigmoid(outputs))
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Final Validation Accuracy: {accuracy:.4f}")


In [ ]:
'_____________________________________'

In [ ]:
import torch
from torchvision import transforms, models
from PIL import Image
import os

# Function to load the trained model
def load_model(model_path='best_model.pth'):
    # Load pre-trained ResNet50 model and modify for binary classification
    model = models.resnet50(pretrained=True)
    model.fc = torch.nn.Linear(model.fc.in_features, 1)  # Output size = 1 for binary classification
    model.load_state_dict(torch.load(model_path))  # Load the saved weights
    model = model.to('cuda' if torch.cuda.is_available() else 'cpu')  # Move to GPU if available
    model.eval()  # Set the model to evaluation mode
    return model

# Function to preprocess the image
def preprocess_image(image_path):
    # Define the transforms (same as used in training)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to 224x224
        transforms.ToTensor(),  # Convert the image to a Tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize based on ImageNet stats
    ])

    # Open the image file and apply the transformations
    image = Image.open(image_path).convert('RGB')  # Ensure the image is in RGB format
    image = transform(image)
    image = image.unsqueeze(0)  # Add a batch dimension
    return image

# Function to make predictions
def predict_anemia(model, image_path):
    # Preprocess the input image
    image = preprocess_image(image_path)
    
    # Move image to the same device as the model (GPU or CPU)
    image = image.to('cuda' if torch.cuda.is_available() else 'cpu')

    # Forward pass through the model to get the prediction
    with torch.no_grad():
        output = model(image)  # Output is a single value

    # Apply sigmoid to the output to get a probability
    probability = torch.sigmoid(output).item()  # Convert the output to a scalar probability

    # If the probability is greater than 0.5, we classify it as "Anemic" (1)
    if probability > 0.5:
        return "Anemic", probability
    else:
        return "Non-Anemic", probability

# Example usage of the functions
if __name__ == '__main__':
    image_path = r'photo1.jpeg'

    # Load the trained model
    model = load_model()

    # Make prediction on the input image
    label, probability = predict_anemia(model, image_path)
    
    # Print the result
    print(f"The person is predicted to be: {label}")
    print(f"Prediction probability: {probability:.4f}")
